<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Merge" data-toc-modified-id="Merge-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Merge</a></span><ul class="toc-item"><li><span><a href="#How-to-merge" data-toc-modified-id="How-to-merge-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>How to merge</a></span></li><li><span><a href="#The-default-&quot;how&quot;-setting-for-merge-is-'inner'!" data-toc-modified-id="The-default-&quot;how&quot;-setting-for-merge-is-'inner'!-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The default "how" setting for merge is 'inner'!</a></span></li><li><span><a href="#Many-to-many-merge:" data-toc-modified-id="Many-to-many-merge:-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Many-to-many merge:</a></span></li></ul></li><li><span><a href="#Join" data-toc-modified-id="Join-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Join</a></span></li><li><span><a href="#Combining-data-with-overlap" data-toc-modified-id="Combining-data-with-overlap-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Combining data with overlap</a></span></li><li><span><a href="#Basic-data-Reshaping:" data-toc-modified-id="Basic-data-Reshaping:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Basic data Reshaping:</a></span><ul class="toc-item"><li><span><a href="#Hierarchical-indexing-:-Stack-and-unstack" data-toc-modified-id="Hierarchical-indexing-:-Stack-and-unstack-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Hierarchical indexing : Stack and unstack</a></span></li><li><span><a href="#Duplication" data-toc-modified-id="Duplication-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Duplication</a></span></li><li><span><a href="#Renaming-axis-indexes:" data-toc-modified-id="Renaming-axis-indexes:-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Renaming axis indexes:</a></span></li></ul></li></ul></div>

# Data Integration with `pandas`

There are four ways to merge/combine between different DataFrames in pandas: concatenating, appending, merging and joining. Each has its own use cases and best practice. 

## Merge

Concat and append have limited capabilities in joining tables through keys and considering the inter-relationship between tables. Merge and join on the other hand combine DataFrames based on a key. According to concepts of relational databases like SQL, there are three types of relationships between tables:

1.	**One-to-one:** When each primary key value relates to only one (or no) record in the related table.

2.	**One-to-many:** When the primary key table contains only one record that relates to none, one, or many records in the related table.

3.	**Many-to-many:** When each record in both tables can relate to any number of records (or no records) in the other table.

We will discuss in the following how merge manages the three types of relationships. The following is an example of using merge for one-to-many relationship between table respresnts customer details and shopping history for each customer. 

In [11]:
import pandas as pd
import numpy as np
customers= pd.DataFrame({'Customer_ID': ['1', '2', '3', '4'],
                      'First_Name': ['A1','A2','A3','A4'],
                    'Last_Name': ['B1', 'B2', 'B3', 'B4']})
shopping_history= pd.DataFrame({'Customer_ID': ['1', '1', '1', '4','5'],
                            'Product_ID':['100','200','300','400','500'],
                      'product': ['Oil','Sugar','Tea','Milk','Eggs']})
merged_df= pd.merge(customers,shopping_history)
print(customers)
print(shopping_history)
print(merged_df)

  Customer_ID First_Name Last_Name
0           1         A1        B1
1           2         A2        B2
2           3         A3        B3
3           4         A4        B4
  Customer_ID Product_ID product
0           1        100     Oil
1           1        200   Sugar
2           1        300     Tea
3           4        400    Milk
4           5        500    Eggs
  Customer_ID First_Name Last_Name Product_ID product
0           1         A1        B1        100     Oil
1           1         A1        B1        200   Sugar
2           1         A1        B1        300     Tea
3           4         A4        B4        400    Milk


In [3]:
merged_df= pd.merge(customers,shopping_history, on="Customer_ID")
print (merged_df)

  Customer_ID First_Name Last_Name Product_ID product
0           1         A1        B1        100     Oil
1           1         A1        B1        200   Sugar
2           1         A1        B1        300     Tea
3           4         A4        B4        400    Milk


### How to merge

When there are no common columns between the DataFrames, you need to specify the key to merge on. Use on, left_on and right_on attributes to define the key in DataFrames.

In [4]:
customers2= pd.DataFrame({'CID': ['1', '2', '3', '4'],
                      'First_Name': ['A1','A2','A3','A4'],
                    'Last_Name': ['B1', 'B2', 'B3', 'B4']})
merged_onkey= pd.merge(customers2,shopping_history,left_on='CID',right_on='Customer_ID')
print (customers2) 
print (shopping_history)
print (merged_onkey)

  CID First_Name Last_Name
0   1         A1        B1
1   2         A2        B2
2   3         A3        B3
3   4         A4        B4
  Customer_ID Product_ID product
0           1        100     Oil
1           1        200   Sugar
2           1        300     Tea
3           4        400    Milk
4           5        500    Eggs
  CID First_Name Last_Name Customer_ID Product_ID product
0   1         A1        B1           1        100     Oil
1   1         A1        B1           1        200   Sugar
2   1         A1        B1           1        300     Tea
3   4         A4        B4           4        400    Milk


### The default "how" setting for merge is 'inner'!

In the joined table, some records are deleted because they do not have a corresponding record in the original DataFrame (such as customer 2,3,5 and Eggs). This happens because the deafult method for merging is the inner join. For zero information loss, you can use outer join instead. The outer join could be full outer (getting full information from both DataFrames), left (only from the left DataFrame) or right (using left, right methods). 

In [5]:
print (customers)
print (shopping_history)
merged_outer= pd.merge(customers,shopping_history, how='outer')
print (merged_outer)
merged_left= pd.merge(customers,shopping_history, how='left')
print (merged_left)
merged_right= pd.merge(shopping_history,customers, how='right')
print (merged_right)

  Customer_ID First_Name Last_Name
0           1         A1        B1
1           2         A2        B2
2           3         A3        B3
3           4         A4        B4
  Customer_ID Product_ID product
0           1        100     Oil
1           1        200   Sugar
2           1        300     Tea
3           4        400    Milk
4           5        500    Eggs
  Customer_ID First_Name Last_Name Product_ID product
0           1         A1        B1        100     Oil
1           1         A1        B1        200   Sugar
2           1         A1        B1        300     Tea
3           2         A2        B2        NaN     NaN
4           3         A3        B3        NaN     NaN
5           4         A4        B4        400    Milk
6           5        NaN       NaN        500    Eggs
  Customer_ID First_Name Last_Name Product_ID product
0           1         A1        B1        100     Oil
1           1         A1        B1        200   Sugar
2           1         A1        B

### Many-to-many merge:
A more complicated relationship to manage using merge method is the many-to-many. Consider the example of customers and products. The customer DataFrame contains information about customers, while products has details about differnt grocery items. One customer can buy non or many products, and any product can be purchased by non or many customers. Pandas manage this relationship using merge method. Like SQL's JOIN clause, pandas.merge allows two DataFrames to be joined on one or more keys. The function provides a series of parameters (on, left_on, right_on, left_index, right_index) allowing you to specify the columns or indexes on which to join. In version 0.17.0. Pandas added the argument indicator. If True, a Categorical-type column called _merge will be added to the output object that takes on values. Here is a simple example.

In [6]:
df1 = pd.DataFrame({'key': ['a', 'a', 'b', 'c', 'a', 'b','c'], 'd1': range(7)})
df2 = pd.DataFrame({'key': ['d', 'd', 'b', 'b', 'b', 'a'], 'd1': range(6)})
print (df1)
print (df2)
merge= pd.merge(df1,df2, how='outer',indicator='True')
print (merge)

  key  d1
0   a   0
1   a   1
2   b   2
3   c   3
4   a   4
5   b   5
6   c   6
  key  d1
0   d   0
1   d   1
2   b   2
3   b   3
4   b   4
5   a   5
   key  d1        True
0    a   0   left_only
1    a   1   left_only
2    b   2        both
3    c   3   left_only
4    a   4   left_only
5    b   5   left_only
6    c   6   left_only
7    d   0  right_only
8    d   1  right_only
9    b   3  right_only
10   b   4  right_only
11   a   5  right_only


In [7]:
print (df1)
print (df2)
merge= pd.merge(df1,df2, how='outer',indicator='True', on='d1')
print (merge)

  key  d1
0   a   0
1   a   1
2   b   2
3   c   3
4   a   4
5   b   5
6   c   6
  key  d1
0   d   0
1   d   1
2   b   2
3   b   3
4   b   4
5   a   5
  key_x  d1 key_y       True
0     a   0     d       both
1     a   1     d       both
2     b   2     b       both
3     c   3     b       both
4     a   4     b       both
5     b   5     a       both
6     c   6   NaN  left_only


In cases that there is a meaningful index in one of the columns, this index can replace the original DataFrame index. By default, set index returns a new DataFrame, so you will have to specify if you would like the changes to occur in place.

## Join

In [8]:
customers.set_index('Customer_ID', inplace=True)
shopping_history.set_index('Customer_ID', inplace=True)
print (customers)
print (shopping_history)

            First_Name Last_Name
Customer_ID                     
1                   A1        B1
2                   A2        B2
3                   A3        B3
4                   A4        B4
            Product_ID product
Customer_ID                   
1                  100     Oil
1                  200   Sugar
1                  300     Tea
4                  400    Milk
5                  500    Eggs


The DataFrame also has a convenient join method for merging on the index. This is used when you have objects with similar row labels, but different columns.

In [9]:
joined= customers.join(shopping_history,how="outer")
print (customers)
print (shopping_history)
print (joined)

            First_Name Last_Name
Customer_ID                     
1                   A1        B1
2                   A2        B2
3                   A3        B3
4                   A4        B4
            Product_ID product
Customer_ID                   
1                  100     Oil
1                  200   Sugar
1                  300     Tea
4                  400    Milk
5                  500    Eggs
            First_Name Last_Name Product_ID product
Customer_ID                                        
1                   A1        B1        100     Oil
1                   A1        B1        200   Sugar
1                   A1        B1        300     Tea
2                   A2        B2        NaN     NaN
3                   A3        B3        NaN     NaN
4                   A4        B4        400    Milk
5                  NaN       NaN        500    Eggs


## Combining data with overlap

We use this method when we want to “patch” values in one object from values for matching indices in the other. Note that this method only takes values from the right DataFrame if they are missing in the left DataFrame. Consider the following example:

In [10]:
import numpy as np
data1= pd.DataFrame([[1,3,np.nan],[np.nan,10,np.nan],[np.nan,5,3]])
data2= pd.DataFrame([[10,np.nan,4],[np.nan,5,3],[2,4,np.nan]])
print (data1)
print (data2)
data= data1.combine_first(data2)
print (data)

     0   1    2
0  1.0   3  NaN
1  NaN  10  NaN
2  NaN   5  3.0
      0    1    2
0  10.0  NaN  4.0
1   NaN  5.0  3.0
2   2.0  4.0  NaN
     0     1    2
0  1.0   3.0  4.0
1  NaN  10.0  3.0
2  2.0   5.0  3.0


## Basic data Reshaping: 

### Hierarchical indexing : Stack and unstack 

Hierarchical indexing provides a more structure way of presenting tabular data. There are two main methods for pivoting data with Hierarchical indexing. 

* stack: this “rotates” or pivots from the columns in the data to the rows
* unstack: this pivots from the rows into the columns

In [14]:
tuples = list(zip(*[['L1_A', 'L1_A', 'L1_B', 'L1_B','L1_C', 'L1_C', 'L1_D', 'L1_D'], 
                    ['L2_1', 'L2_2', 'L2_1','L2_2','L2_1', 'L2_2', 'L2_1','L2_2']]))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
data = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])
print(data)
result = data.stack()
print(result)

                     A         B
first second                    
L1_A  L2_1   -0.885529 -2.050589
      L2_2    0.900622 -1.049695
L1_B  L2_1    0.302075 -0.415061
      L2_2    0.336935 -0.934590
L1_C  L2_1    0.298216  0.834307
      L2_2   -1.675568 -0.130639
L1_D  L2_1    1.203295  0.463200
      L2_2   -0.115449  1.655784
first  second   
L1_A   L2_1    A   -0.885529
               B   -2.050589
       L2_2    A    0.900622
               B   -1.049695
L1_B   L2_1    A    0.302075
               B   -0.415061
       L2_2    A    0.336935
               B   -0.934590
L1_C   L2_1    A    0.298216
               B    0.834307
       L2_2    A   -1.675568
               B   -0.130639
L1_D   L2_1    A    1.203295
               B    0.463200
       L2_2    A   -0.115449
               B    1.655784
dtype: float64


In [13]:
data.unstack()

A                   B          
second      L2_1      L2_2      L2_1      L2_2
first                                         
L1_A    0.531221  0.035309 -0.208840 -0.604975
L1_B    0.244420 -0.335794 -0.159944 -0.331012
L1_C    0.938382  0.447665  0.502007  1.830525
L1_D    1.193371  1.163797 -0.245722 -0.392397

### Duplication

The DataFrame method 'duplicated' returns a Boolean Series indicating whether each row is a duplicate or not:

In [16]:
data.duplicated()

first  second
L1_A   L2_1      False
       L2_2      False
L1_B   L2_1      False
       L2_2      False
L1_C   L2_1      False
       L2_2      False
L1_D   L2_1      False
       L2_2      False
dtype: bool

Relatedly, 'drop_duplicates' returns a DataFrame where the duplicated array is without duplicated rows.

In [17]:
data = pd.DataFrame({'k1': ['Milk'] * 3 + ['Cheese'] * 4,  'k2': [1, 1, 2, 3, 3, 4, 4]})
print (data)
data.duplicated()
cleandata= data.drop_duplicates()
print (cleandata)

       k1  k2
0    Milk   1
1    Milk   1
2    Milk   2
3  Cheese   3
4  Cheese   3
5  Cheese   4
6  Cheese   4
       k1  k2
0    Milk   1
2    Milk   2
3  Cheese   3
5  Cheese   4


As you may have noticed, applying drop_duplicates on the whole DataFrame considrs all othe columns together. We can alternatively specify which column we want to capture the duplication at.

In [18]:
v1= data.drop_duplicates(['k1'])
print (v1)
v2=  data.drop_duplicates(['k2'])
print (v2)

       k1  k2
0    Milk   1
3  Cheese   3
       k1  k2
0    Milk   1
2    Milk   2
3  Cheese   3
5  Cheese   4


### Renaming axis indexes: 

Pandas enable modifying the current attribute name using map or renaming methods. Example is as follows:

In [19]:
idf= pd.DataFrame(np.arange(12).reshape((3, 4)), 
                  index=['nsw', 'vic', 'tasmania'],
                  columns=['day1', 'day2', 'day3', 'day4'])
print (idf)
idf.index.map(str.upper)
idf.rename(index=str.upper, columns=str.upper)

          day1  day2  day3  day4
nsw          0     1     2     3
vic          4     5     6     7
tasmania     8     9    10    11


,DAY1,DAY2,DAY3,DAY4
NSW,0,1,2,3
VIC,4,5,6,7
TASMANIA,8,9,10,11


In [20]:
idf

,day1,day2,day3,day4
nsw,0,1,2,3
vic,4,5,6,7
tasmania,8,9,10,11


Please note rename keeps the original value of the data unchanged. To change the original DataFrame, you need to do that in place. You can also rename a subset of columns.

In [21]:
#idf.index=idf.index.map(str.upper)
#OR
idf.rename(index=str.upper, columns=str.upper, inplace=True)
idf

,DAY1,DAY2,DAY3,DAY4
NSW,0,1,2,3
VIC,4,5,6,7
TASMANIA,8,9,10,11
